In [1]:
import os

os.chdir("../..")
os.getcwd()

'/home/INTEXSOFT/maksim.stupakevich/work/ds_practice'

In [2]:
import pickle
from collections import defaultdict

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from Levenshtein import editops
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm

In [3]:
train_df = pd.read_csv("data/enzyme/train.csv", index_col="seq_id")

In [4]:
updated_df = pd.read_csv("data/enzyme/train_updates_20220929.csv", index_col="seq_id")

In [5]:
updated_df = updated_df.drop(["data_source"], axis=1).dropna()

In [6]:
train_df.update(updated_df)

In [7]:
train_df

,protein_sequence,pH,data_source,tm
seq_id,,,,
0,AAAAKAAALALLGEAPEVVDIWLPAGWRQPFRVFRLERKGDGVLVG...,7.0,doi.org/10.1038/s41592-020-0801-4,75.7
1,AAADGEPLHNEEERAGAGQVGRSLPQESEEQRTGSRPRRRRDLGSR...,7.0,doi.org/10.1038/s41592-020-0801-4,50.5
2,AAAFSTPRATSYRILSSAGSGSTRADAPQVRRLHTTRDLLAKDYYA...,7.0,doi.org/10.1038/s41592-020-0801-4,40.5
3,AAASGLRTAIPAQPLRHLLQPAPRPCLRPFGLLSVRAGSARRSGLL...,7.0,doi.org/10.1038/s41592-020-0801-4,47.2
4,AAATKSGPRRQSQGASVRTFTPFYFLVEPVDTLSVRGSSVILNCSA...,7.0,doi.org/10.1038/s41592-020-0801-4,49.5
...,...,...,...,...
31385,YYMYSGGGSALAAGGGGAGRKGDWNDIDSIKKKDLHHSRGDEKAQG...,7.0,doi.org/10.1038/s41592-020-0801-4,51.8
31386,YYNDQHRLSSYSVETAMFLSWERAIVKPGAMFKKAVIGFNCNVDLI...,7.0,doi.org/10.1038/s41592-020-0801-4,37.2
31387,YYQRTLGAELLYKISFGEMPKSAQDSAENCPSGMQFPDTAIAHANV...,7.0,doi.org/10.1038/s41592-020-0801-4,64.6


In [8]:
def get_count(groups):
    count = 0
    for k, v in groups.items():
        count += len(v)
    return count

In [9]:
def group(df):
    groups = defaultdict(list)
    proteins = df["protein_sequence"].unique().tolist()
    with tqdm(total=len(proteins)) as pbar:
        while proteins:
            protein = proteins.pop(0)
            idxs_to_del = []
            for i, el in enumerate(proteins):
                similarity = editops(protein, el)
                if len(similarity) <= 100:
                    groups[protein].append(el)
                    idxs_to_del.append(i)
            [proteins.pop(el) for el in list(reversed(idxs_to_del))]
            pbar.update(len(groups[protein]) + 1)

    return {i: [k, *v] for i, (k, v) in enumerate(groups.items())}

In [10]:
groups = group(train_df)

100%|███████████████████████████████████| 28981/28981 [3:53:11<00:00,  2.07it/s]


In [11]:
get_count(groups)

28981

In [12]:
len(groups)

21277

In [13]:
len({k: v for k, v in groups.items() if len(v) >= 5})

154

In [14]:
with open("data/enzyme/train_groups_4_all.pkl", "wb") as f:
    pickle.dump(groups, f)

In [15]:
with open("data/enzyme/train_groups_4.pkl", "wb") as f:
    pickle.dump({k: v for k, v in groups.items() if len(v) > 1}, f)

In [9]:
with open("data/enzyme/train_groups_3.pkl", "rb") as f:
    groups = pickle.load(f)

In [114]:
train_df["protein_len"] = train_df["protein_sequence"].str.len()

In [115]:
train_df.head()

,protein_sequence,pH,data_source,tm,protein_len
seq_id,,,,,
0,AAAAKAAALALLGEAPEVVDIWLPAGWRQPFRVFRLERKGDGVLVG...,7.0,doi.org/10.1038/s41592-020-0801-4,75.7,341
1,AAADGEPLHNEEERAGAGQVGRSLPQESEEQRTGSRPRRRRDLGSR...,7.0,doi.org/10.1038/s41592-020-0801-4,50.5,286
2,AAAFSTPRATSYRILSSAGSGSTRADAPQVRRLHTTRDLLAKDYYA...,7.0,doi.org/10.1038/s41592-020-0801-4,40.5,497
3,AAASGLRTAIPAQPLRHLLQPAPRPCLRPFGLLSVRAGSARRSGLL...,7.0,doi.org/10.1038/s41592-020-0801-4,47.2,265
4,AAATKSGPRRQSQGASVRTFTPFYFLVEPVDTLSVRGSSVILNCSA...,7.0,doi.org/10.1038/s41592-020-0801-4,49.5,1451


In [116]:
lenghts = train_df.groupby("protein_len").count()[["protein_sequence"]].index.tolist()

In [117]:
GROUP_LEN_DIFFERENCE = 1
MIN_GROUP_SIZE = 5
GROUP_AA_DIFFERENCE = 1

In [118]:
len_groups = {}
for str_len in lenghts:
    len_group = train_df[str_len == train_df["protein_len"]][
        "protein_sequence"
    ].tolist()
    if len(len_group) >= 2:
        len_groups[str_len] = len_group

In [119]:
groups = defaultdict(dict)
with tqdm(total=len(len_groups)) as pbar:

    for group_len, proteins in len_groups.items():
        protein_groups = defaultdict(list)
        while proteins:
            protein = proteins.pop(0)
            idxs_to_del = []
            for i, el in enumerate(proteins):
                similarity = editops(protein, el)
                if len(similarity) <= GROUP_AA_DIFFERENCE:
                    protein_groups[protein].append(el)
                    idxs_to_del.append(i)
            [proteins.pop(el) for el in list(reversed(idxs_to_del))]
        idxs_to_del = []
        for protein_group, proteins in protein_groups.items():
            if len(proteins) < MIN_GROUP_SIZE:
                idxs_to_del.append(protein_group)
        [protein_groups.pop(el) for el in list(reversed(idxs_to_del))]
        if protein_groups:
            groups[group_len] = protein_groups
        pbar.update(1)

100%|██████████████████████████████████████| 1394/1394 [00:12<00:00, 107.49it/s]


In [120]:
def merge_groups(groups):
    new_groups = {}
    start_len = len(groups)
    i = 0
    while groups:
        proteins = groups.pop(list(groups.keys())[0])
        protein = proteins[0]

        idxs_to_del = []
        for k, v in groups.items():
            similar = editops(protein, v[0])
            if len(similar) <= 5:
                proteins.extend(v)
                idxs_to_del.append(k)
        [groups.pop(el) for el in idxs_to_del]
        new_groups[i] = proteins
        i += 1

    return new_groups

In [121]:
len(groups)

48

In [122]:
new_groups = {}
number = 0
with tqdm(total=len(groups)) as pbar:
    for group_len, len_groups in groups.items():
        for _, merged_group in merge_groups(len_groups).items():
            if len(merged_group) >= MIN_GROUP_SIZE:
                new_groups[number] = merged_group
                number += 1
        pbar.update(len(idxs_to_del) + 1)

100%|████████████████████████████████████████| 48/48 [00:00<00:00, 13574.71it/s]


In [123]:
len(new_groups)

54

In [126]:
with open("data/enzyme/train_groups_5.pkl", "wb") as f:
    pickle.dump(new_groups, f)

In [10]:
def get_group(protein, groups):
    for group_num, group_proteins in groups.items():
        if protein in group_proteins:
            return group_num
    return -1

In [32]:
train_df["group"] = train_df["protein_sequence"].apply(
    lambda x: get_group(
        x,
        groups,
    )
)

In [33]:
train_df[train_df["group"] != -1].head()

,protein_sequence,pH,data_source,tm,group
seq_id,,,,,
5,AACFWRRTVIPKPPFRGISTTSARSTVMPAWVIDKYGKNEVLRFTQ...,7.0,doi.org/10.1038/s41592-020-0801-4,48.4,5
6,AACFWRRTVIPKPPFRGISTTSARSTVMPAWVIDKYGKNEVLRFTQ...,7.0,doi.org/10.1038/s41592-020-0801-4,45.7,5
20,AAMNPEYDYLFKLLLIGDSGVGKSCLLLRFADDTYTESYISTIGVD...,7.0,doi.org/10.1038/s41592-020-0801-4,42.5,19
31,AATSVFCGSSHRQLHHAVIPHGKGGRSSVSGVVATVFGATGFLGRY...,7.0,doi.org/10.1038/s41592-020-0801-4,40.8,30
40,ACPDSNDKEIRGFCFKFVVQKMAYNDARNWCHYQNPVGPSYLAVVG...,7.0,doi.org/10.1038/s41592-020-0801-4,45.7,39


In [13]:
def get_wildtypes(groups, method="mode"):
    wildtypes = {}
    if method == "mode":
        for group_num, group_proteins in groups.items():
            wildtypes[group_num] = stats.mode(groups[group_num])[0][0]
    elif method == "editops":
        for group_num, group_proteins in groups.items():
            differenсies = {}
            for cur_protein in group_proteins:
                differenсies[cur_protein] = 0
                for protein in group_proteins:
                    differenсies[cur_protein] += len(editops(cur_protein, protein))
            wildtypes[group_num] = min(differenсies, key=differenсies.get)
    else:
        raise TypeError
    return wildtypes

In [34]:
all_grouped_proteins = {
    k: v
    for k, v in train_df.groupby("group")["protein_sequence"]
    .apply(list)
    .to_dict()
    .items()
    if k != -1
}

In [37]:
mode_wildtypes = get_wildtypes(
    all_grouped_proteins,
)

In [38]:
editops_wildtypes = get_wildtypes(
    groups,
    method="editops",
)

In [39]:
train_df["wildtype"] = train_df.apply(
    lambda x: editops_wildtypes[x["group"]] if x["group"] != -1 else np.nan,
    axis=1,
)

In [40]:
train_df["mode_wildtype"] = train_df.apply(
    lambda x: mode_wildtypes[x["group"]] if x["group"] != -1 else np.nan,
    axis=1,
)

In [41]:
train_df.head()

,protein_sequence,pH,data_source,tm,group,wildtype,mode_wildtype
seq_id,,,,,,,
0,AAAAKAAALALLGEAPEVVDIWLPAGWRQPFRVFRLERKGDGVLVG...,7.0,doi.org/10.1038/s41592-020-0801-4,75.7,-1,NaN,NaN
1,AAADGEPLHNEEERAGAGQVGRSLPQESEEQRTGSRPRRRRDLGSR...,7.0,doi.org/10.1038/s41592-020-0801-4,50.5,-1,NaN,NaN
2,AAAFSTPRATSYRILSSAGSGSTRADAPQVRRLHTTRDLLAKDYYA...,7.0,doi.org/10.1038/s41592-020-0801-4,40.5,-1,NaN,NaN
3,AAASGLRTAIPAQPLRHLLQPAPRPCLRPFGLLSVRAGSARRSGLL...,7.0,doi.org/10.1038/s41592-020-0801-4,47.2,-1,NaN,NaN
4,AAATKSGPRRQSQGASVRTFTPFYFLVEPVDTLSVRGSSVILNCSA...,7.0,doi.org/10.1038/s41592-020-0801-4,49.5,-1,NaN,NaN


In [42]:
train_df[train_df["group"] != -1].head()

,protein_sequence,pH,data_source,tm,group,wildtype,mode_wildtype
seq_id,,,,,,,
5,AACFWRRTVIPKPPFRGISTTSARSTVMPAWVIDKYGKNEVLRFTQ...,7.0,doi.org/10.1038/s41592-020-0801-4,48.4,5,AACFWRRTVIPKPPFRGISTTSARSTVMPAWVIDKYGKNEVLRFTQ...,AACFWRRTVIPKPPFRGISTTSARSTVMPAWVIDKYGKNEVLRFTQ...
6,AACFWRRTVIPKPPFRGISTTSARSTVMPAWVIDKYGKNEVLRFTQ...,7.0,doi.org/10.1038/s41592-020-0801-4,45.7,5,AACFWRRTVIPKPPFRGISTTSARSTVMPAWVIDKYGKNEVLRFTQ...,AACFWRRTVIPKPPFRGISTTSARSTVMPAWVIDKYGKNEVLRFTQ...
20,AAMNPEYDYLFKLLLIGDSGVGKSCLLLRFADDTYTESYISTIGVD...,7.0,doi.org/10.1038/s41592-020-0801-4,42.5,19,MNPEYDYLFKLLLIGDSGVGKSCLLLRFADDSYLDSYISTIGVDFK...,AAMNPEYDYLFKLLLIGDSGVGKSCLLLRFADDTYTESYISTIGVD...
31,AATSVFCGSSHRQLHHAVIPHGKGGRSSVSGVVATVFGATGFLGRY...,7.0,doi.org/10.1038/s41592-020-0801-4,40.8,30,AATSVFCGSSHRQLHHAVIPHGKGGRSSVSGVVATVFGATGFLGRY...,AATSVFCGSSHRQLHHAVIPHGKGGRSSVSGVVATVFGATGFLGRY...
40,ACPDSNDKEIRGFCFKFVVQKMAYNDARNWCHYQNPVGPSYLAVVG...,7.0,doi.org/10.1038/s41592-020-0801-4,45.7,39,ACPDSNDKEIRGFCFKFVVQKMTYNDARNWCHYQNPVGPSYLAVVG...,ACPDSNDKEIRGFCFKFVVQKMAYNDARNWCHYQNPVGPSYLAVVG...


In [43]:
train_df.to_csv("data/enzyme/train_with_wildtypes.csv", index=False)

In [44]:
editops(train_df.loc[28856, "wildtype"], train_df.loc[28856, "protein_sequence"])

[('replace', 28, 28)]

In [74]:
train_df["group"].value_counts()

-1        21863
 13290      816
 12987      783
 10919      308
 13859      211
          ...  
 7530         2
 19840        2
 11660        2
 9613         2
 13937        2
Name: group, Length: 1344, dtype: int64

In [84]:
train_df.loc[train_df["group"] == 13859, "wildtype"]

seq_id
19480    MNQSVSSLPEKDIQYQLHPYTNARLHQELGPLIIERGEGIYVYDDQ...
19481    MNQSVSSLPEKDIQYQLHPYTNARLHQELGPLIIERGEGIYVYDDQ...
19482    MNQSVSSLPEKDIQYQLHPYTNARLHQELGPLIIERGEGIYVYDDQ...
19483    MNQSVSSLPEKDIQYQLHPYTNARLHQELGPLIIERGEGIYVYDDQ...
19484    MNQSVSSLPEKDIQYQLHPYTNARLHQELGPLIIERGEGIYVYDDQ...
                               ...                        
19686    MNQSVSSLPEKDIQYQLHPYTNARLHQELGPLIIERGEGIYVYDDQ...
19687    MNQSVSSLPEKDIQYQLHPYTNARLHQELGPLIIERGEGIYVYDDQ...
19688    MNQSVSSLPEKDIQYQLHPYTNARLHQELGPLIIERGEGIYVYDDQ...
19689    MNQSVSSLPEKDIQYQLHPYTNARLHQELGPLIIERGEGIYVYDDQ...
19690    MNQSVSSLPEKDIQYQLHPYTNARLHQELGPLIIERGEGIYVYDDQ...
Name: wildtype, Length: 211, dtype: object

In [85]:
train_df.loc[19480, "wildtype"]

'MNQSVSSLPEKDIQYQLHPYTNARLHQELGPLIIERGEGIYVYDDQGKGYIEAMAGLWSAALGFSNQRLIKAAEQQFNTLPFYHLFSHKSHRPSIELAEKLIEMAPVPMSKVFFTNSGSEANDTVVKMVWYLNNALGKPAKKKFISRVNGYHGITVASASLTGLPGNQRGFDLPLPGFLHVGCPHHYRFALAGESEEHFADRLAVELEQKILAEGPETIAAFIGEPLMGAGGVIVPPRTYWEKIQKVCRKYDILVIADEVICGFGRTGQMFGSQTFGIQPDIMVLSKQLSSSYQPIAAILINAPVFEGIADQSQALGALGHGFTGSGHPVATAVALENLKIIEEESLVEHAAQMGQLLRSGLQHFIDHPLVGEIRGCGLIAAVELVGDRVSKAPYQALGTLGRYMAGRAQEHGMITRAMGDAVAFCPPLIVNEQEVGMIVERFARALADTTQWVG'

In [88]:
train_df["group"]

seq_id
0       -1
1       -1
2       -1
3       -1
4       -1
        ..
31385   -1
31386   -1
31387   -1
31388   -1
31389   -1
Name: group, Length: 31390, dtype: int64